In [1]:
import onnx
import torch
import numpy as np
import pandas as pd  
import copy
import pprint
import json

from onnx2pytorch import ConvertModel
from torch import nn

import sys
sys.path.append('/Users/khoanguyen-cp/gmu/network_properties')

from algorithms.iterative_relaxation import IterativeRelaxation
from algorithms.decision_procedure import MarabouCoreDP
from algorithms.decision_tree import DecisionTree

from models.test_models import ProphecyPaperNetwork, TestModel
from models.acasxu_1_1 import Acasxu1_1
from models.utils import attach_relu_activation_hook, attach_layer_output_hook, get_layers_info
from models.utils import turn_bool_activation_to_int, turn_bool_activation_to_str


torch.set_printoptions(precision=8) 

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
model = Acasxu1_1()
model.load_state_dict(torch.load('./models/acasxu_1_1.pt'))
_act_handles, activation_signature = attach_relu_activation_hook(model)

In [3]:
model

Acasxu1_1(
  (fc0): Linear(in_features=5, out_features=50, bias=True)
  (relu0): ReLU()
  (fc1): Linear(in_features=50, out_features=50, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=50, out_features=50, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=50, out_features=50, bias=True)
  (relu4): ReLU()
  (fc5): Linear(in_features=50, out_features=50, bias=True)
  (relu5): ReLU()
  (output_layer): Linear(in_features=50, out_features=5, bias=True)
)

In [4]:
from network_properties.read_vnnlib import read_vnnlib
from pathlib import Path
vnnlib_path = Path("./network_properties/prop_1.vnnlib") # acasxu
# vnnlib_path = Path("./network_properties/prop_0_0.03.vnnlib") # MNIST

input_ranges, specifications = read_vnnlib(vnnlib_path)[0]
print(input_ranges)
print(specifications)

5 inputs and 5 outputs in vnnlib
[[0.6, 0.679857769], [-0.5, 0.5], [-0.5, 0.5], [0.45, 0.5], [-0.5, -0.45]]
[(array([[-1.,  0.,  0.,  0.,  0.]]), array([-3.99112565]))]


## Test input property

In [5]:
_act_handles, activation_signature = attach_relu_activation_hook(model)

In [6]:
input_data = [[0.62, 0.1, 0.2, 0.47, -0.48]]
X = torch.tensor(input_data, dtype=torch.float)
_logits = model(X)

activation_signature = turn_bool_activation_to_str(activation_signature)
for layer_name, activations in activation_signature.items():
  activation_signature[layer_name] = activations[0]
  
print(activation_signature)

{'relu0': ['ON', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'ON', 'ON', 'ON', 'OFF', 'ON', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'ON', 'ON', 'OFF', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 'ON', 'OFF', 'OFF', 'ON', 'ON', 'ON', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'ON'], 'relu1': ['OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'ON', 'ON', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF'], 'relu2': ['ON', 'ON', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'ON', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'ON', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 'OFF'

In [7]:
dp = MarabouCoreDP()
dp.solve(activation_signature, model, input_ranges, specifications)

['unsat',
 {},

## 0. Prepare ACASXU dataset

In [8]:
# !wget https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/clusterinACAS_0_short.csv -O ./datasets/clusterinACAS_0_shrt.csv

In [9]:
acas_train = np.empty([384221,5],dtype=float)
acas_train_labels = np.zeros(384221,dtype=int)

def read_inputs_from_file(inputFile):
  global acas_train, acas_train_labels, num
  with open(inputFile) as f:
    lines = f.readlines()
    print(len(lines), "examples")
    acas_train = np.empty([len(lines),5],dtype=float)
    acas_train_labels = np.zeros(len(lines),dtype=int)

    for l in range(len(lines)):
      # This is to remove the useless 1 at the start of each string. Not sure why that's there.
      k = [float(stringIn) for stringIn in lines[l].split(',')] 
      
      # acas_train[l+num] = np.zeros(5,dtype=float) 
      # we're asuming that everything is 2D for now. The 1 is just to keep numpy happy.
      if len(k) > 5:
        lab = int(k[5])
        #if ((lab == 0) or (lab == 2)):
        #  lab = 0
        #else:
        #  lab = 1
        acas_train_labels[l+num] = lab

      count = 0
      for i in range(0,5):
        #print(count)
        acas_train[l+num][i] = k[i]
        #print(k[i])

In [10]:
num = 0
read_inputs_from_file('./datasets/clusterinACAS_0_shrt.csv')
print(acas_train.shape)
print(acas_train_labels.shape)
acas_train[:5]

384221 examples
(384221, 5)
(384221,)


array([[-0.320142, -0.5     , -0.5     , -0.5     , -0.5     ],
       [-0.295234, -0.5     , -0.5     , -0.5     , -0.5     ],
       [-0.240207, -0.5     , -0.5     , -0.5     , -0.5     ],
       [-0.208943, -0.5     , -0.5     , -0.5     , -0.5     ],
       [-0.183636, -0.5     , -0.5     , -0.5     , -0.5     ]])

In [11]:
def create_df(inputs, predicted_labels, true_labels, activation_signature):
  data = []
  for index, input_data in enumerate(inputs):
    data_point = { 
      "input": input_data, 
      "true_label": true_labels[index], 
      "predicted_label": predicted_labels[index].item(),
    }
    data_point_full_signature = {} 
    for name, layer_activation in activation_signature.items():
      data_point[name] = json.dumps(layer_activation[index])
      data_point_full_signature[name] = layer_activation[index]
      
    data_point['full_signature'] = json.dumps(data_point_full_signature)
    data.append(data_point)

  return pd.DataFrame(data)

outputs = model(torch.tensor(acas_train, dtype=torch.float32))
predicted_labels = torch.argmin(outputs, dim=1)
activation_signature = turn_bool_activation_to_int(activation_signature, to_list=True)
df = create_df(acas_train, predicted_labels, acas_train_labels, activation_signature)
df.head(10)

,input,true_label,predicted_label,relu0,relu1,relu2,relu3,relu4,relu5,full_signature
0,"[-0.320142, -0.5, -0.5, -0.5, -0.5]",3,3,"[1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...","[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, ...","[0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","{""relu0"": [1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,..."
1,"[-0.295234, -0.5, -0.5, -0.5, -0.5]",1,1,"[1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...","[1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...","[0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{""relu0"": [1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,..."
2,"[-0.240207, -0.5, -0.5, -0.5, -0.5]",0,0,"[1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, ...","[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{""relu0"": [1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,..."
3,"[-0.208943, -0.5, -0.5, -0.5, -0.5]",0,0,"[1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, ...","[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{""relu0"": [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,..."
4,"[-0.183636, -0.5, -0.5, -0.5, -0.5]",0,0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, ...","[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{""relu0"": [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,..."
5,"[-0.10156, -0.5, -0.5, -0.5, -0.5]",0,0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, ...","[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{""relu0"": [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,..."
6,"[-0.000732, -0.5, -0.5, -0.5, -0.5]",0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{""relu0"": [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,..."
7,"[0.100096, -0.5, -0.5, -0.5, -0.5]",0,0,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{""relu0"": [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,..."
8,"[0.200941, -0.5, -0.5, -0.5, -0.5]",0,0,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{""relu0"": [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,..."
9,"[0.301769, -0.5, -0.5, -0.5, -0.5]",0,0,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 

In [12]:
# test accuracy
len(df[df["predicted_label"] == df['true_label']]) / (len(df))

0.7663037678835878

## 1. Input properties

### 1.1 With DP 

In [13]:
# Prepare candidate input property for each class 
# For example: specification for output = class 0, meaning constraints for NOT class 0
# since the result is argmin, this would mean we'll encode constraints for y0 NOT being the min value
# i.e. y1 <= y0 or y2 <= y0 or y3 <= y0 or y4 <= y0
specification_for_classes = {
  0: [(np.array([[-1, 1, 0, 0, 0]]), np.array([0])),
      (np.array([[-1, 0, 1, 0, 0]]), np.array([0])),
      (np.array([[-1, 0, 0, 1, 0]]), np.array([0])),
      (np.array([[-1, 0, 0, 0, 1]]), np.array([0]))],
  
  1: [(np.array([[1, -1, 0, 0, 0]]), np.array([0])),
      (np.array([[0, -1, 1, 0, 0]]), np.array([0])),
      (np.array([[0, -1, 0, 1, 0]]), np.array([0])),
      (np.array([[0, -1, 0, 0, 1]]), np.array([0]))],
  
  2: [(np.array([[1, 0, -1, 0, 0]]), np.array([0])),
      (np.array([[0, 1, -1, 0, 0]]), np.array([0])),
      (np.array([[0, 0, -1, 1, 0]]), np.array([0])),
      (np.array([[0, 0, -1, 0, 1]]), np.array([0]))],
  
  3: [(np.array([[1, 0, 0, -1, 0]]), np.array([0])),
      (np.array([[0, 1, 0, -1, 0]]), np.array([0])),
      (np.array([[0, 0, 1, -1, 0]]), np.array([0])),
      (np.array([[0, 0, 0, -1, 1]]), np.array([0]))],
  
  4: [(np.array([[1, 0, 0, 0, -1]]), np.array([0])),
      (np.array([[0, 1, 0, 0, -1]]), np.array([0])),
      (np.array([[0, 0, 1, 0, -1]]), np.array([0])),
      (np.array([[0, 0, 0, 1, -1]]), np.array([0]))],
}

candidates = {}
input_ranges = {}
samples = {}

for y_class in range(5):
  # Group the DataFrame by 'full_signature' column and get the group sizes
  # Get the signature with the highest support
  class_df = df[df['predicted_label'] == y_class]
  group_sizes = class_df.groupby('full_signature').size()
  activation_with_max_support = group_sizes.idxmax()
  activation_pattern = json.loads(activation_with_max_support)
  
  # Turn activation from 0 and 1 to ON and OFF to match DP's format
  for layer, activation in activation_pattern.items():
    activation_pattern[layer] = ["ON" if val == 1 else "OFF" for val in activation]
  candidates[y_class] = activation_pattern
  
  # Find input ranges for the candidate property
  ranges = []
  support_df = class_df[class_df['full_signature'] == activation_with_max_support]
  for i in range(5):
    min_val = support_df['input'].apply(lambda x: x[i]).min()
    max_val = support_df['input'].apply(lambda x: x[i]).max()
    ranges.append([min_val, max_val])
  input_ranges[y_class] = ranges
  
  # Sample a data point from support
  sample_row = support_df.sample()
  assert str(sample_row.get('full_signature').item()) == str(activation_with_max_support)
  samples[y_class] = sample_row.get('input').item()
  
# pprint.pprint("CANDIDATE PROPERTIES:")
# pprint.pprint(candidates)

# pprint.pprint("INPUT RANGES:")
# pprint.pprint(input_ranges)

# pprint.pprint("SAMPLES:")
# pprint.pprint(samples)

In [14]:
input_ranges

{0: [[0.100096, 0.478202],
  [-0.449995, -0.250001],
  [-0.5, -0.349998],
  [-0.214282, 0.071427],
  [0.25, 0.5]],
 1: [[-0.295234, -0.295234],
  [-0.5, -0.400004],
  [-0.300007, -0.199994],
  [-0.214282, -0.214282],
  [-0.5, -0.5]],
 2: [[-0.295234, -0.295234],
  [0.300007, 0.349998],
  [0.449995, 0.5],
  [0.357182, 0.5],
  [-0.5, -0.5]],
 3: [[-0.320142, -0.320142],
  [-0.5, -0.449995],
  [-0.449995, -0.349998],
  [0.071427, 0.071427],
  [0.0, 0.0]],
 4: [[-0.295234, -0.295234],
  [0.449995, 0.449995],
  [-0.5, -0.400004],
  [0.357182, 0.5],
  [-0.5, -0.5]]}

In [15]:
samples

{0: array([ 0.150527, -0.349998, -0.449995,  0.071427,  0.375   ]),
 1: array([-0.295234, -0.5     , -0.199994, -0.214282, -0.5     ]),
 2: array([-0.295234,  0.300007,  0.5     ,  0.5     , -0.5     ]),
 3: array([-0.320142, -0.449995, -0.349998,  0.071427,  0.      ]),
 4: array([-0.295234,  0.449995, -0.5     ,  0.5     , -0.5     ])}

In [16]:
for y_class in range(5):
  activation_pattern = candidates[y_class]
  x_ranges = input_ranges[y_class]
  specification = specification_for_classes[y_class]
  status, counter_example, _, _ = dp.solve(activation_pattern, model, x_ranges, specification)
  if status == 'unsat':
    print(f"{activation_pattern} is an input property for class {y_class}\n\n")


{'relu0': ['ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 'OFF', 'ON', 'ON', 'ON', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'ON'], 'relu1': ['OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF'], 'relu2': ['OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'ON', 'OFF', 'ON', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 

### 1.2 With Iterative Relaxation

In [ ]:
iterative_relaxation = IterativeRelaxation()

for y_class in range(5):
  x_ranges = input_ranges[y_class]
  input_sample = np.array([samples[y_class]])
  specification = specification_for_classes[y_class]
  input_property = iterative_relaxation.call(model, input_sample, x_ranges, specification)
  print(f"input property for class {y_class}: {input_property}\n\n")

unconstrained_layer: relu5
{'relu0': ['ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 'OFF', 'ON', 'ON', 'ON', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'ON'], 'relu1': ['OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF'], 'relu2': ['OFF', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'ON', 'OFF', 'ON', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'ON', 'OFF', 'OFF', 'OFF', 'OFF', 'ON', 'OFF', 'OFF',

--- unconstraining neuron 43 in critical layer
--- neuron needed
--- unconstraining neuron 44 in critical layer
--- unconstraining neuron 45 in critical layer
--- neuron needed
--- unconstraining neuron 46 in critical layer
--- unconstraining neuron 47 in critical layer


## 2. Layer patterns as interpolants

### 2.1 Find candidates with Decision Tree

In [ ]:
# relu_layers = ['relu0', 'relu1', 'relu2', 'relu3', 'relu4']
# labels = [0,1,2,3,4]
# for name, module in list(model.named_modules()):
#   if isinstance(module, torch.nn.ReLU):
#     relu_layers.append(name)
# relu_layers

In [ ]:
# for layer_name in relu_layers: 
#   for chosen_label in [0,1,2,3,4]:
#     df['satisfies_postcon'] = np.where(df['predicted_label'] == chosen_label, 1, -1)
#     decision_tree = DecisionTree(df, X_col=layer_name, Y_col="satisfies_postcon")
#     leaves_with_activation_pattern = decision_tree.get_potential_layer_properties()

In [ ]:
# df['satisfies_postcon'] = np.where(df['predicted_label'] == 3, 1, -1)
# decision_tree = DecisionTree(df, X_col='relu4', Y_col="satisfies_postcon")
# leaves_with_activation_pattern = decision_tree.get_potential_layer_properties()

# num_of_patterns = len(leaves_with_activation_pattern)
# print(f"num_of_patterns: {num_of_patterns}")

# total_support = sum(candidate['support'] for candidate in leaves_with_activation_pattern)
# print(f"total_support: {total_support}")

# top_5_supports = [candidate['support'] for candidate in leaves_with_activation_pattern[:5]]
# print(f"top_5_supports: {top_5_supports}")